In [1]:
import os
from collections import defaultdict
from random import shuffle
import xml.etree.ElementTree as ET
from tqdm import tqdm
import pandas as pd
import pybboxes as pbx
import shutil

In [2]:
from ultralytics import YOLO

In [3]:
annotations_path = "../data/annotations/"
images_path = "../data/images/"

In [4]:
annotations = os.listdir(annotations_path)
images = os.listdir(images_path)

In [5]:
img_name_list = []
width_list = []
height_list = []
label_list = []
x_min = []
y_min = []
x_max = []
y_max = []

In [6]:
for i in tqdm(range(len(annotations))):
    tree = ET.parse(os.path.join(annotations_path, annotations[i]))
    root = tree.getroot()
    img_name = root.find("filename").text

    size = root.find("size")
    width = int(size.find("width").text)
    height = int(size.find("height").text)

    for group in root.findall("object"):
        label = group.find("name").text
        bbox = group.find("bndbox")
        xmin = int(bbox.find("xmin").text)
        ymin = int(bbox.find("ymin").text)
        xmax = int(bbox.find("xmax").text)
        ymax = int(bbox.find("ymax").text)

        img_name_list.append(img_name)
        width_list.append(width)
        height_list.append(height)
        x_min.append(xmin)
        y_min.append(ymin)
        x_max.append(xmax)
        y_max.append(ymax)
        label_list.append(label)

100%|██████████| 877/877 [00:00<00:00, 6800.21it/s]


In [7]:
dataset = pd.DataFrame(
    {
        "img_name": img_name_list,
        "width": width_list,
        "height": height_list,
        "x_min": x_min,
        "y_min": y_min,
        "x_max": x_max,
        "y_max": y_max,
        "label": label_list,
    }
)
dataset.head()

,img_name,width,height,x_min,y_min,x_max,y_max,label
0,road0.png,267,400,98,62,208,232,trafficlight
1,road1.png,400,283,154,63,258,281,trafficlight
2,road10.png,400,267,106,3,244,263,trafficlight
3,road100.png,400,385,35,5,363,326,speedlimit
4,road101.png,400,200,195,7,392,194,speedlimit


In [8]:
classes = dataset["label"].unique().tolist()
class_training = {idx: label for idx, label in enumerate(classes)}
classes = {label: idx for idx, label in enumerate(classes)}
classes

{'trafficlight': 0, 'speedlimit': 1, 'crosswalk': 2, 'stop': 3}

In [9]:
dataset["class"] = dataset["label"].map(classes)

In [10]:
dataset.head()

,img_name,width,height,x_min,y_min,x_max,y_max,label,class
0,road0.png,267,400,98,62,208,232,trafficlight,0
1,road1.png,400,283,154,63,258,281,trafficlight,0
2,road10.png,400,267,106,3,244,263,trafficlight,0
3,road100.png,400,385,35,5,363,326,speedlimit,1
4,road101.png,400,200,195,7,392,194,speedlimit,1


In [11]:
imgs = defaultdict(list)
for idx, dt in tqdm(dataset.iterrows()):
    sample_label_list = []
    img_name = dt["img_name"]
    xmin = int(dt["x_min"])
    ymin = int(dt["y_min"])
    xmax = int(dt["x_max"])
    ymax = int(dt["y_max"])
    class_num = dt["class"]
    W, H = int(dt["width"]), int(dt["height"])

    voc_bbox = (int(xmin), int(ymin), int(xmax), int(ymax))

    x_center, y_center, w, h = pbx.convert_bbox(
        voc_bbox, from_type="voc", to_type="yolo", image_size=(W, H)
    )

    sample_label_list.append(str(class_num))
    sample_label_list.append(str(x_center))
    sample_label_list.append(str(y_center))
    sample_label_list.append(str(w))
    sample_label_list.append(str(h))
    line = " ".join(sample_label_list)

    imgs[img_name].append(line)

1244it [00:00, 8987.85it/s]


In [12]:
labels_dir = "../data/labels"
if os.path.exists(labels_dir):
    shutil.rmtree(labels_dir)
os.mkdir(labels_dir)

In [13]:
for img_name, lines in imgs.items():
    img_name = img_name.split('.')[0]
    with open(f'{labels_dir}/{img_name}.txt', 'w') as f:
        for line in lines:
            f.write(line)
            f.write('\n')

In [14]:
train_dir = "../data/train/"
val_dir = "../data/val"
labels_path = "../data/labels/"

In [15]:
if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

if os.path.exists(val_dir):
    shutil.rmtree(val_dir)

os.mkdir(train_dir)
os.mkdir(val_dir)

# train, val each containing images and labels folders
os.mkdir(train_dir + "/images")
os.mkdir(train_dir + "/labels")
os.mkdir(val_dir + "/images")
os.mkdir(val_dir + "/labels")

In [16]:
files = os.listdir(images_path)
shuffle(files)

In [17]:
def split(files, ratio):
    elements = len(files)
    middle = int(elements * ratio)
    return [files[:middle], files[middle:]]

def copy_files(images_path, labels_path, destination_path, files):
    
    for file_name in files:
        file_name = file_name.split('.')[0]

        src = images_path + f'/{file_name}.png'
        dst = destination_path + '/images'
        shutil.copy(src, dst)

        src = labels_path + f'/{file_name}.txt'
        dst = destination_path + '/labels'
        shutil.copy(src, dst)

In [18]:
train_ratio = 0.75
train_files, val_files = split(files, train_ratio)

root = 'data/traffic_sign_data'

copy_files(images_path, labels_path, train_dir, train_files)
copy_files(images_path, labels_path, val_dir, val_files)

In [19]:
with open(f'../data/sign_data.yaml', 'w') as f:
    f.write('train: ../../data/train/images\n')
    f.write('val: ../../data/val/images\n')
    f.write('nc: 4\n')
    f.write(f"names: {class_training}")

In [20]:
model = YOLO("yolov8n.pt")
results = model.train(
    data="../data/sign_data.yaml", epochs=10, imgsz=320
)  # train the model
results = model.val()  # evaluate model performance on the validation set

New https://pypi.org/project/ultralytics/8.0.234 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.233 🚀 Python-3.9.13 torch-2.1.0+cpu CPU (AMD Ryzen 7 6800H with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=../data/sign_data.yaml, epochs=10, time=None, patience=50, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fa

train: Scanning C:\Users\ADMIN\OneDrive - EPITA\Project-Demo\road-sign-detection\data\train\labels... 657 images, 0 backgrounds, 0 corrupt: 100%|██████████| 657/657 [00:00<00:00, 897.65it/s]

train: New cache created: C:\Users\ADMIN\OneDrive - EPITA\Project-Demo\road-sign-detection\data\train\labels.cache



val: Scanning C:\Users\ADMIN\OneDrive - EPITA\Project-Demo\road-sign-detection\data\val\labels... 220 images, 0 backgrounds, 0 corrupt: 100%|██████████| 220/220 [00:00<00:00, 791.02it/s]

val: New cache created: C:\Users\ADMIN\OneDrive - EPITA\Project-Demo\road-sign-detection\data\val\labels.cache


...


10 epochs completed in 0.103 hours.
Optimizer stripped from runs\detect\train3\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train3\weights\best.pt, 6.2MB

Validating runs\detect\train3\weights\best.pt...
Ultralytics YOLOv8.0.233 🚀 Python-3.9.13 torch-2.1.0+cpu CPU (AMD Ryzen 7 6800H with Radeon Graphics)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:04<00:00,  1.69it/s]


                   all        220        300      0.873      0.804      0.872      0.717
          trafficlight        220         36      0.684      0.556       0.68      0.467
            speedlimit        220        206      0.999      0.956      0.984       0.83
             crosswalk        220         39      0.914      0.795      0.849      0.689
                  stop        220         19      0.896       0.91      0.975      0.884
Speed: 0.2ms preprocess, 12.2ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs\detect\train3
Ultralytics YOLOv8.0.233 🚀 Python-3.9.13 torch-2.1.0+cpu CPU (AMD Ryzen 7 6800H with Radeon Graphics)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\ADMIN\OneDrive - EPITA\Project-Demo\road-sign-detection\data\val\labels.cache... 220 images, 0 backgrounds, 0 corrupt: 100%|██████████| 220/220 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:04<00:00,  3.46it/s]


                   all        220        300      0.873      0.804      0.872      0.717
          trafficlight        220         36      0.684      0.556      0.678      0.466
            speedlimit        220        206      0.999      0.956      0.984       0.83
             crosswalk        220         39      0.914      0.795      0.849      0.689
                  stop        220         19      0.896      0.909      0.975      0.883
Speed: 0.2ms preprocess, 11.8ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\train32
